# ETL Costa Oeste de los Estados Unidos 

La Costa Oeste de EEUU incluyen a "California, Oregón, Washington, Alaska, Arizona y Nevada"
En este trabajo no se tendrá en cuenta a Alaska ya que ....

In [23]:
# Se importan librerias
import pandas as pd
import os
import ast
import json

## Trabajando con Json

### Review Estados

In [ ]:
def leer_contenido_carpeta(ruta_carpeta):
    """
    Lee el contenido de una carpeta y devuelve una lista de diccionarios con los datos de los archivos JSON.

    Args:
        ruta_carpeta (str): Ruta de la carpeta a leer.

    Returns:
        list: Lista de diccionarios con los datos de los archivos JSON.
    """
    
    try:
        # Verificar si la ruta es un directorio
        if os.path.isdir(ruta_carpeta):
            # Obtener lista de archivos y directorios en la carpeta
            contenido = os.listdir(ruta_carpeta)
            
            # Lista para almacenar datos de todos los archivos JSON en la carpeta
            df = []
            
            # Recorrer la lista de archivos y directorios
            for item in contenido:
                item_ruta = os.path.join(ruta_carpeta, item)
                
                # Si es un archivo, leer si es JSON
                if os.path.isfile(item_ruta) and item.endswith('.json'):
                    print("Leyendo el archivo:", item_ruta)
                    
                    # Leer y procesar el archivo JSON
                    with open(item_ruta, 'r', encoding='utf-8') as f:
                        for line in f:
                            df.append(json.loads(line))
                
                # Si es un directorio, llamar recursivamente a la función
                elif os.path.isdir(item_ruta):
                    print(f"Entrando en carpeta: {item_ruta}")
                    subcontenido = leer_contenido_carpeta(item_ruta)
                    if subcontenido is not None:
                        df.extend(subcontenido)
            
            # Convertir la lista de diccionarios a DataFrame
            df_final = pd.DataFrame(df)
            
            # Especificar la ruta donde se guardarán los archivos CSV
            ruta_guardado = '../csv/Google Maps/reviews-estados'
            
            # Obtener el nombre de la carpeta que contiene los archivos JSON
            nombre_carpeta = os.path.basename(ruta_carpeta)
            
            # Guardar el DataFrame como CSV en la ruta especificada
            nombre_archivo_final = os.path.join(ruta_guardado, nombre_carpeta + '.csv')
            df_final.to_csv(nombre_archivo_final, index=False)
            print("Se guardó con éxito en:", nombre_archivo_final)                    
                    
        else:
            print(f"La ruta '{ruta_carpeta}' no es un directorio válido.")
    except Exception as e:
        print(f"Error al leer el contenido de la carpeta: {e}")


In [ ]:
leer_contenido_carpeta('../Data/Google Maps/reviews-estados/')

### Metadata

In [ ]:
import os
def leer_contenido_carpeta(ruta_carpeta):
    """ Lee el contenido de una carpeta y devuelve una lista de archivos y directorios.

    Args:
        ruta_carpeta (str): Ruta de la carpeta a leer.

    Returns:
        list: Lista de archivos y directorios.
    """
    
    try:
        # Verificar si la ruta es un directorio
        if os.path.isdir(ruta_carpeta):
            # Obtener lista de archivos y directorios en la carpeta
            contenido = os.listdir(ruta_carpeta)
            
            # Lista para almacenar datos de todos los archivos JSON en la carpeta
            df = []
            
            # Recorrer la lista de archivos y directorios
            for item in contenido:
                item_ruta = os.path.join(ruta_carpeta, item)
                
                # Si es un archivo, leer si es JSON
                if os.path.isfile(item_ruta) and item.endswith('.json'):
                    print("Leyendo el archivo:", item_ruta)
                    
                    # Leer y procesar el archivo JSON
                    with open(item_ruta, 'r', encoding='utf-8') as f:
                        for line in f:
                            df.append(json.loads(line))
            
            # Convertir la lista de diccionarios a DataFrame
            df_final = pd.DataFrame(df)
            
            # Especificar la ruta donde se guardarán los archivos CSV
            ruta_guardado = '../csv/Google Maps/metadata-sitios'
            
            # Obtener el nombre de la carpeta que contiene los archivos JSON
            nombre_carpeta = os.path.basename(ruta_carpeta)
            
            # Guardar el DataFrame como CSV en la ruta especificada
            nombre_archivo_final = os.path.join(ruta_guardado, nombre_carpeta + '.csv')
            df_final.to_csv(nombre_archivo_final, index=False)
            print("Se guardó con éxito en:", nombre_archivo_final)                    
                    
        else:
            print(f"La ruta '{ruta_carpeta}' no es un directorio válido.")
    except Exception as e:
        print(f"Error al leer el contenido de la carpeta: {e}")


In [ ]:
leer_contenido_carpeta('../Data/Google Maps/metadata-sitios/')

### Yelp User Parquet dividido

In [ ]:
def dividir_archivo_parquet_en_csv(ruta_archivo_parquet, carpeta_destino_csv, num_filas_por_archivo=450000):
    """ Divide un archivo Parquet en partes más pequeñas y las guarda como CSV.
    El archivo Parquet se lee y se divide en partes más pequeñas. Cada parte se guarda como un archivo CSV.
    El número de filas por archivo se puede establecer como un parámetro opcional.
    Por defecto, el número de filas por archivo es de 450000.
    El archivo CSV se guarda en la carpeta especificada como parámetro.
    Si la carpeta no existe, se crea.
    Si el archivo Parquet no existe, se levanta una excepción.
    Si el archivo Parquet está vacío, se levanta una excepción.
    Si el archivo Parquet tiene menos filas que el número de filas por archivo, se levanta una excepción.
    """
    
    # Leer el archivo Parquet
    archivo_grande = pd.read_parquet(ruta_archivo_parquet)

    # Dividir el archivo en partes más pequeñas
    num_archivos = (len(archivo_grande) // num_filas_por_archivo) + 1

    for i in range(num_archivos):
        inicio = i * num_filas_por_archivo
        fin = min((i + 1) * num_filas_por_archivo, len(archivo_grande))  
        archivo_pequeno = archivo_grande.iloc[inicio:fin]
        
        # Guardar el archivo como CSV
        nombre_archivo_csv = f'archivo_pequeno_{i}.csv'
        ruta_completa_csv = os.path.join(carpeta_destino_csv, nombre_archivo_csv)
        archivo_pequeno.to_csv(ruta_completa_csv, index=False)


In [ ]:
ruta_archivo_grande = '../Data/Yelp/user.parquet'
carpeta_archivos_pequenos_csv = '../csv/Yelp/user/'
dividir_archivo_parquet_en_csv(ruta_archivo_grande, carpeta_archivos_pequenos_csv)

### Yelp Jsons

In [ ]:
def procesar_archivo_json(ruta_archivo_json, ruta_guardado):
    
    """ 
    Procesa un archivo JSON y lo guarda en un archivo CSV.
    Args:
        ruta_archivo_json (str): Ruta del archivo JSON a procesar.
        ruta_guardado (str): Ruta donde se guardará el archivo CSV resultante.
    Returns:
        None.
    """
    try:
        if os.path.isfile(ruta_archivo_json) and ruta_archivo_json.endswith('.json'):
            nombre_archivo = os.path.splitext(os.path.basename(ruta_archivo_json))[0]
            
            # Verificar si el nombre del archivo es diferente de "review"
            if nombre_archivo != "review":
                print("Leyendo el archivo:", ruta_archivo_json)
                with open(ruta_archivo_json, 'r', encoding='utf-8') as f:
                    data = [json.loads(line) for line in f]

                # Convertir la lista de diccionarios a DataFrame
                df = pd.DataFrame(data)

                # Guardar el DataFrame como CSV en la ruta especificada
                nombre_archivo_csv = os.path.join(ruta_guardado, nombre_archivo + '.csv')
                df.to_csv(nombre_archivo_csv, index=False)
                print("Se guardó con éxito en:", nombre_archivo_csv)
    except Exception as e:
        print(f"Error al procesar el archivo JSON: {e}")

def leer_contenido_carpeta(ruta_carpeta, ruta_guardado):
    try:
        # Verificar si la ruta es un directorio
        if os.path.isdir(ruta_carpeta):
            # Obtener lista de archivos y directorios en la carpeta
            contenido = os.listdir(ruta_carpeta)
            
            # Recorrer la lista de archivos y directorios
            for item in contenido:
                item_ruta = os.path.join(ruta_carpeta, item)
                
                # Si es un archivo, procesar si es JSON
                if os.path.isfile(item_ruta):
                    procesar_archivo_json(item_ruta, ruta_guardado)
                
                # Si es un directorio, llamar recursivamente a la función
                elif os.path.isdir(item_ruta):
                    print(f"Entrando en carpeta: {item_ruta}")
                    leer_contenido_carpeta(item_ruta, ruta_guardado)
        else:
            print(f"La ruta '{ruta_carpeta}' no es un directorio válido.")
    except Exception as e:
        print(f"Error al leer el contenido de la carpeta: {e}")

In [ ]:
ruta_carpeta = '../Data/Yelp/'
ruta_guardado = '../csv/Yelp/'  
leer_contenido_carpeta(ruta_carpeta, ruta_guardado)

### Yelp review devidido

In [ ]:
def dividir_archivo_json_en_csv(ruta_json_grande, carpeta_archivos_pequenos, tamano_archivo_pequeno=450000):
    
    """ 
    Dividir un archivo JSON grande en archivos CSV pequeños.

    Parámetros:
        ruta_json_grande (str): Ruta del archivo JSON grande.
        carpeta_archivos_pequenos (str): Ruta de la carpeta donde se guardarán los archivos CSV pequeños.
        tamano_archivo_pequeno (int): Tamaño máximo de cada archivo CSV pequeño (en líneas). Por defecto, 450000.

    Retorna:
        None. Los archivos CSV pequeños se guardan en la carpeta especificada.
    """
    # Función para escribir un fragmento JSON en un archivo CSV
    def escribir_fragmento_csv(numero_archivo, fragmento):
        nombre_archivo = f'fragmento_{numero_archivo}.csv'
        ruta_archivo = os.path.join(carpeta_archivos_pequenos, nombre_archivo)
        with open(ruta_archivo, 'w', newline='', encoding='utf-8') as archivo_csv:
            escritor_csv = csv.DictWriter(archivo_csv, fieldnames=fragmento[0].keys())
            escritor_csv.writeheader()
            escritor_csv.writerows(fragmento)

    # Leer el archivo JSON grande y dividirlo en fragmentos más pequeños
    with open(ruta_json_grande, 'r', encoding='utf-8') as archivo_grande:
        fragmento_actual = []
        numero_archivo = 1
        for linea in archivo_grande:
            objeto_json = json.loads(linea)
            fragmento_actual.append(objeto_json)
            if len(fragmento_actual) == tamano_archivo_pequeno:
                escribir_fragmento_csv(numero_archivo, fragmento_actual)
                numero_archivo += 1
                fragmento_actual = []
        # Escribir el último fragmento si es necesario
        if fragmento_actual:
            escribir_fragmento_csv(numero_archivo, fragmento_actual)

In [ ]:
ruta_json_grande = '../Data/Yelp/review.json'
carpeta_archivos_pequenos = '../csv/Yelp/review/'
dividir_archivo_json_en_csv(ruta_json_grande, carpeta_archivos_pequenos)

### Business

In [ ]:
# Lee el archivo pickle desde la ubicación especificada
buis = pd.read_pickle('../Data/Yelp/business.pkl')

# Guarda el DataFrame como un archivo CSV en la ubicación deseada
ruta_guardado = '../csv/Yelp/business.csv'
buis.to_csv(ruta_guardado, index=False)

## Trabajando con CSV

### Estados

Se eliminan las comunas pics y resp ya que están prácticamente vacías

In [25]:
def limpiar_csv_estados(ruta_carpeta):
    """ Función para limpiar los archivos CSV de la carpeta de estados de USA.
    Elimina las columnas "pics" y "resp" del archivo CSV.
    Guarda el archivo CSV en la carpeta "Data_limpia/Google_Maps/review_estados" con el mismo nombre.
    Imprime la cantidad de Nulos por columna del archivo CSV.
    Llama a la función recursiva para limpiar los directorios contenidos en la carpeta.
    Recibe la ruta de la carpeta como parámetro.
    
    Args: ruta_carpeta (str): Ruta de la carpeta que contiene los archivos csv.

    """

    
    try:
        # Verificar si la carpeta existe
        if os.path.isdir(ruta_carpeta):
            # Obtener lista de archivos y directorios en la carpeta
            archivos = os.listdir(ruta_carpeta)
            
            # Recorrer la lista de archivos y directorios
            for archivo in archivos:
                # Verificar si el archivo es un directorio
                if os.path.isdir(os.path.join(ruta_carpeta, archivo)):
                    # Llamar a la función recursiva para limpiar el directorio
                    limpiar_csv_estados(os.path.join(ruta_carpeta, archivo))
                else:
                    # Verificar si el archivo es un CSV
                    if archivo.endswith('.csv'):
                        #Abrir csv y eliminar columnas innecesarias
                        print(f'Entrando al archivo : {archivo}')
                        df = pd.read_csv(os.path.join(ruta_carpeta, archivo))
                        df.drop(columns=['pics','resp'], inplace=True)
                        #Guardar csv
                        ruta_guardado = '../Data_limpia/Google_Maps/review_estados/'
                        df.to_csv(os.path.join(ruta_guardado, archivo), index=False)
                        print(f'Se eliminaron las columnas "pics" y "resp" del archivo {archivo} y se guardó con éxito en: {os.path.join(ruta_guardado, archivo)}')
                        print(f'La cantidad de Nulos por columna del archivo {archivo} es la siguiente:\n{df.isnull().sum()}')
                    else:
                        print(f'El archivo {archivo} no es un CSV, se pudo leer')
        else:
            print(f"La carpeta {ruta_carpeta} no existe") 
                
    except Exception as e:
        print(f"Error al leer el contenido de la carpeta: {e}")

In [26]:
limpiar_csv_estados('../csv/Google Maps/costa_oeste/')

Entrando al archivo : review-Arizona.csv
Se eliminaron las columnas "pics" y "resp" del archivo review-Arizona.csv y se guardó con éxito en: ../Data_limpia/Google_Maps/review_estados/review-Arizona.csv
La cantidad de Nulos por columna del archivo review-Arizona.csv es la siguiente:
user_id         0
name            7
time            0
rating          0
text       832205
gmap_id         0
dtype: int64
Entrando al archivo : review-California.csv
Se eliminaron las columnas "pics" y "resp" del archivo review-California.csv y se guardó con éxito en: ../Data_limpia/Google_Maps/review_estados/review-California.csv
La cantidad de Nulos por columna del archivo review-California.csv es la siguiente:
user_id          0
name             7
time             0
rating           0
text       1170997
gmap_id          0
dtype: int64
Entrando al archivo : review-Nevada.csv
Se eliminaron las columnas "pics" y "resp" del archivo review-Nevada.csv y se guardó con éxito en: ../Data_limpia/Google_Maps/review_e

### Metadata y Business

In [27]:
# Se extraen datos
df_metatada = pd.read_csv('../csv/Google Maps/metadata-sitios/metadata-sitios.csv')
df_business = pd.read_csv('../csv/Yelp/business.csv')

C:\Users\gerar\AppData\Local\Temp\ipykernel_12524\718445187.py:3: DtypeWarning: Columns (14,15,16,17,18,19,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_business = pd.read_csv('../csv/Yelp/business.csv')


Se crea la columna coordenadas y se elimina latitud y longitud para un manejo más conveniente

In [28]:
df_metatada['coordenadas'] = df_metatada.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
df_metatada.drop(columns=['latitude', 'longitude'], inplace=True)
df_metatada['coordenadas'] = df_metatada['coordenadas'].apply(lambda x: ', '.join(map(str,x)))

df_business['coordenadas'] = df_business.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
df_business.drop(columns=['latitude', 'longitude'], inplace=True)
df_business['coordenadas'] = df_business['coordenadas'].apply(lambda x: ', '.join(map(str,x)))

Se abre el contenido de la columna "category" y se tratan los nulos de esta columna

In [30]:
# Reemplazar NaN con listas vacías
df_metatada['category'] = df_metatada['category'].fillna('[]')

# Convertir las cadenas de caracteres en listas de Python
df_metatada['category'] = df_metatada['category'].apply(ast.literal_eval)

# Aplicar explode
df_metatada = df_metatada.explode('category')

Se extraen solo las columnas necesarias y se unen a través de la columna "coordenadas" 

In [32]:
df_metatada_v2 = df_metatada[['name','category','avg_rating','coordenadas','num_of_reviews']]
df_business_v2 = df_business[['business_id','city','coordenadas','state','stars','review_count']]

df_buis_metadata = pd.merge(df_metatada_v2, df_business_v2, on='coordenadas', how='inner')

Como solo se va a trabajar con 5 estados, se seleccionan solo las filas que tengan alguno de estos estados

In [36]:
BuMet_Costa_Oeste = df_buis_metadata[df_buis_metadata['state'].isin(['CA', 'OR','WA','AZ','NV'])]

Se guardan los datos en csv para su posterior uso

In [ ]:
BuMet_Costa_Oeste.to_csv('../Data_limpia/Google_Maps/Buismet_Costa_Oeste.csv', index=False)

### Reviews

Se eliminan las columnas useful, funny y cool por su falta de relevancia para el proyecto final, se extraen solo las filas que coincidan con los locales que se va a trabajar aclaras en el csv "Buismet_Costa_Oeste"

In [4]:
import os
def eleccion_review(ruta_carpeta):
    """ Función para limpiar los archivos de reviews.
    
    Args: ruta_carpeta (str): Ruta de la carpeta que contiene los archivos csv.

    """
    
    try:
        # verifica si la carpeta existe
        if os.path.isdir(ruta_carpeta):
            # obtiene una lista de archivos en la carpeta
            archivos = os.listdir(ruta_carpeta)
            
            # Lista para almacenar todos los DataFrames fusionados
            lista_df = []
            
            # itera sobre la lista de archivos
            for archivo in archivos:
                # verifica si el archivo es un directorio
                if os.path.isdir(os.path.join(ruta_carpeta, archivo)):
                    # llama recursivamente a la función para limpiar el directorio
                    eleccion_review(os.path.join(ruta_carpeta, archivo))
                else:
                    #verifica si el archivo es un csv
                    if archivo.endswith('.csv'):
                        # abre el csv y elimina las columnas innecesarias
                        print(f'Leyendo el archivo {archivo}...')
                        df = pd.read_csv(os.path.join(ruta_carpeta, archivo))
                        df.drop(columns=['useful','funny','cool'], inplace=True)
                        
                        # Convierte la columna de date al formato de date y hora si no está en ese formato
                        df['date'] = pd.to_datetime(df['date'])
                        
                        # Filtra las filas para incluir solo aquellas con fechas a partir de 2015
                        df = df[df['date'].dt.year >= 2015]
                        
                        # Lee el archivo "Buismet_Costa_Oeste" y elimina las columnas innecesarias
                        # compara con el archivo "Buismet_Costa_Oeste" y solo se guardan las filas que coincidan ambos con business_id
                        meta = pd.read_csv('../Data_limpia/Google_Maps/Buismet_Costa_Oeste.csv')
                        df_final = pd.merge(df, meta[['business_id']], on='business_id', how='inner')
                        # Agrega el DataFrame fusionado a la lista
                        lista_df.append(df_final)
                        print(f'Se eliminaron las columnas \"useful\", \"funny\", \"cool\" del archivo {archivo} y solo se extrajo las filas del año 2015 en adelante')
                    else:
                        print(f'El archivo {archivo} no es un CSV, se pudo leer')
                        
            # Fusiona todos los DataFrames en uno solo
            df_final_total = pd.concat(lista_df, ignore_index=True)
            
            # Guarda el DataFrame fusionado en un solo archivo CSV
            ruta_guardado = '../Data_limpia/Yelp/reviews/'
            df_final_total.to_csv(os.path.join(ruta_guardado, 'reviews_desde_2015.csv'), index=False)
            print(f'Se guardaron todos los archivos fusionados en: {os.path.join(ruta_guardado, "reviews_desde_2015.csv")}')
            
        else:
            print(f"La carpeta {ruta_carpeta} no existe")   
    except Exception as e:
        print(f"Error al leer el contenido de la carpeta: {e}")

In [5]:
eleccion_review('../csv/Yelp/review/')

Leyendo el archivo fragmento_1.csv...
Se eliminaron las columnas "useful", "funny", "cool" del archivo fragmento_1.csv y solo se extrajo las filas del año 2015 en adelante
Leyendo el archivo fragmento_10.csv...
Se eliminaron las columnas "useful", "funny", "cool" del archivo fragmento_10.csv y solo se extrajo las filas del año 2015 en adelante
Leyendo el archivo fragmento_11.csv...
Se eliminaron las columnas "useful", "funny", "cool" del archivo fragmento_11.csv y solo se extrajo las filas del año 2015 en adelante
Leyendo el archivo fragmento_12.csv...
Se eliminaron las columnas "useful", "funny", "cool" del archivo fragmento_12.csv y solo se extrajo las filas del año 2015 en adelante
Leyendo el archivo fragmento_13.csv...
Se eliminaron las columnas "useful", "funny", "cool" del archivo fragmento_13.csv y solo se extrajo las filas del año 2015 en adelante
Leyendo el archivo fragmento_14.csv...
Se eliminaron las columnas "useful", "funny", "cool" del archivo fragmento_14.csv y solo se e

### Users

Se eliminan las columnas ('review_count', 'yelping_since', 'useful', 'funny', 'cool', 'elite', 'friends', 'fans','average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer' y 'compliment_photos') por su falta de relevancia para el proyecto final, se extraen solo las filas que coincidan con los "user_id" que se va a trabajar aclaras en el csv "reviews_desde_2015.csv"

In [15]:
def eleccion_users(ruta_carpeta):
    
    """ 
    Función que se encarga de limpiar los archivos csv de la carpeta "users"
    y fusionarlos con el archivo "reviews_desde_2015.csv" para obtener un
    archivo limpio de usuarios.
    
    Args: ruta_carpeta (str): Ruta de la carpeta que contiene los archivos csv.

    """
    try:
        # verifica si la carpeta existe
        if os.path.isdir(ruta_carpeta):
            # obtiene una lista de archivos en la carpeta
            archivos = os.listdir(ruta_carpeta)
            
            # Lista para almacenar todos los DataFrames fusionados
            lista_df = []
            
            # itera sobre la lista de archivos
            for archivo in archivos:
                # verifica si el archivo es un directorio
                if os.path.isdir(os.path.join(ruta_carpeta, archivo)):
                    # llama recursivamente a la función para limpiar el directorio
                    eleccion_users(os.path.join(ruta_carpeta, archivo))
                else:
                    #verifica si el archivo es un csv
                    if archivo.endswith('.csv'):
                        # abre el csv y elimina las columnas innecesarias
                        print(f'Leyendo el archivo {archivo}...')
                        df = pd.read_csv(os.path.join(ruta_carpeta, archivo))
                        
                        print('Eliminando columnas innecesarias...')
                        df.drop(columns=['review_count','yelping_since','useful','funny','cool','elite','friends','fans','average_stars','compliment_hot','compliment_more','compliment_profile','compliment_cute','compliment_list','compliment_note','compliment_plain','compliment_cool','compliment_funny','compliment_writer','compliment_photos'], inplace=True)
                        # compara con el archivo "reviews_desde_2015.csv" y solo se guardan las filas que coincidan ambos con user_id
                        rev = pd.read_csv('../Data_limpia/Yelp/reviews/reviews_desde_2015.csv')
                        df_final = pd.merge(df, rev[['user_id']], on='user_id', how='inner')
                        
                        # Agrega el DataFrame fusionado a la lista
                        lista_df.append(df_final)
                        
                    else:
                        print(f'El archivo {archivo} no es un CSV, se pudo leer')
                        
            # Fusiona todos los DataFrames en uno solo
            df_final_total = pd.concat(lista_df, ignore_index=True)
            
            # Guarda el DataFrame fusionado en un solo archivo CSV
            ruta_guardado = '../Data_limpia/Yelp/users/'
            df_final_total.to_csv(os.path.join(ruta_guardado, 'users_desde_2015.csv'), index=False)
            print(f'Se guardaron todos los archivos fusionados en: {os.path.join(ruta_guardado, "users_fusionados.csv")}')
            
        else:
            print(f"La carpeta {ruta_carpeta} no existe")   
    except Exception as e:
        print(f"Error al leer el contenido de la carpeta: {e}")

In [16]:
eleccion_users('../csv/Yelp/user/')

Leyendo el archivo archivo_pequeno_0.csv...
Eliminando columnas innecesarias...
Leyendo el archivo archivo_pequeno_1.csv...
Eliminando columnas innecesarias...
Leyendo el archivo archivo_pequeno_2.csv...
Eliminando columnas innecesarias...
Leyendo el archivo archivo_pequeno_3.csv...


C:\Users\gerar\AppData\Local\Temp\ipykernel_14204\250418535.py:22: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(ruta_carpeta, archivo))


Eliminando columnas innecesarias...
Leyendo el archivo archivo_pequeno_4.csv...
Eliminando columnas innecesarias...
Se guardaron todos los archivos fusionados en: ../Data_limpia/Yelp/users/users_fusionados.csv


In [17]:
reviews = pd.read_csv('../Data_limpia/Yelp/reviews/reviews_desde_2015.csv')
users = pd.read_csv('../Data_limpia/Yelp/users/users_desde_2015.csv')

In [18]:
reviews_users_desde_2015 = pd.merge(users, reviews, on='user_id', how='inner')

In [20]:
reviews_users_desde_2015

,user_id,name,review_id,business_id,stars,text,date
0,IpLRJY4CP3fXtlEd8Y4GFQ,Robyn,rqpFNOI5LIYKI484x4SHpA,_vi0nhfx0jSlIOp8PzD8QQ,5.0,Heard about this place via NBC-10 and the Elle...,2015-01-14 11:26:56
1,IpLRJY4CP3fXtlEd8Y4GFQ,Robyn,rqpFNOI5LIYKI484x4SHpA,_vi0nhfx0jSlIOp8PzD8QQ,5.0,Heard about this place via NBC-10 and the Elle...,2015-01-14 11:26:56
2,IpLRJY4CP3fXtlEd8Y4GFQ,Robyn,rqpFNOI5LIYKI484x4SHpA,_vi0nhfx0jSlIOp8PzD8QQ,5.0,Heard about this place via NBC-10 and the Elle...,2015-01-14 11:26:56
3,IpLRJY4CP3fXtlEd8Y4GFQ,Robyn,sMqbUjo9g3DJVSQGa8tnpw,kI_8RT6-IoVxD-EgXmyFvg,2.0,"The food was good enough, but they weren't ver...",2016-06-17 16:01:51
4,IpLRJY4CP3fXtlEd8Y4GFQ,Robyn,sMqbUjo9g3DJVSQGa8tnpw,kI_8RT6-IoVxD-EgXmyFvg,2.0,"The food was good enough, but they weren't ver...",2016-06-17 16:01:51
...,...,...,...,...,...,...,...
2563866,7iUcYua1GtbIcHjTqIiG3Q,Linda,R69rsGF2BchLAa7a7s_KMg,xmgLT96LecWFLQBaX-mB1w,5.0,Truckee River Rock developed our landscaping p...,2015-12-29 18:50:53
2563867,f9Lm5TrNfVX1JmVXIbtCOg,Bernae,tUZZoZuDUF-RKBLALwC-qA,o4MAhdTefZhXHgDvx1pSvA,5.0,Great location and the staff are friendly and ...,2016-06-03 23:41:01
2563868,f9Lm5TrNfVX1JmVXIbtCOg,Bernae,tUZZoZuDUF-RKBLALwC-qA,o4MAhdTefZhXHgDvx1pSvA,5.0,Great location and the staff are friendly and ...,2016-06-03 23:41:01
2563869,f9Lm5TrNfVX1JmVXIbtCOg,Bernae,tUZZoZuDUF-RKBLALwC-qA,o4MAhdTefZhXHgDvx1pSvA,5.0,Great location and the staff are friendly and ...,2016-06-03 23:41:01


In [21]:
filas_duplicadas = reviews_users_desde_2015.duplicated(keep=False)  # keep=False marca todas las filas duplicadas como True
total_filas_duplicadas = filas_duplicadas.sum()

In [22]:
total_filas_duplicadas

2551382